<a href="https://colab.research.google.com/github/stanleyeze/Unification-Algorithm/blob/master/UnificationAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unification Algorithm

AI unification algorithm unifies two functions that are equal. The unification is done with occurs check.

*   Variable are uppercase
*   Function symboles are in lowercase letters

## Example

1.   unify a and X.
      X = a

      yes, can unify


2.   unify Y and X
     Y = X
     yes, can unify

3.   unify X and f(Y)
     X = f(Y)
     yes, can unify

4.   unify X and g(X)
     no, cannot unify in occurs check. This would cause an infinit loop because      X exist in g(X). 

To learn more about this visit -> https://docs.racket-lang.org/racklog/unification.html



# Implementation

In [0]:
# Provides four classes to represent substituition, variables, constants, and expressions.
# Variable and Constant names start with upper and lower case letters, respectively.
# Expressions are trees with Variables and/or Constants as leaves.
# Each internal node of an expression includes an operator and an argument list. Operator stands for the name
# of the function/expression and arguments can be variables, constants or other expressions
#
#The Substitution class makes the substitution between a variable and other variables/functions (Expressions)
#
# The parse_expression(s): Parses a string s and returns a corresponding expression object.
# Assumes arguments are separated by commas and zero or more whitespace.

class Substitution:
	def __init__(self, variable, replacement):
		self.variable = variable
		self.replacement = replacement

	def __str__(self):
		return str(self.variable) + " = " + str(self.replacement)

		
class Variable:
    def __init__(self, variable_name):
        """
        Initialize a variable with the given name.
        The name should start with an upper-case letter.
        """
        if variable_name[0].islower(): raise (Exception("Variable name starting with lower-case!"))
        self.variable_name = variable_name

    def __eq__(self, other):
        """
        Checks whether self and other represent the same variable.
        Two variables are the same if they have the same name.
        """
        return isinstance(other, Variable) and self.variable_name == other.variable_name

    def __ne__(self, other):
        """
        Checks whether self and other represent different variables.
        """
        return not self.__eq__(other)

    def __str__(self):
        """
        Returns a string representation of the variable.
        """
        return self.variable_name

    def __repr__(self):
        """
        Returns a string representation of the variable.
        """
        return str(self)

    def __hash__(self):
        """
        Produces a hash of the variable so it can be stored in a dictionary.
        """
        return str(self).__hash__()

    def occurs_in(self, other):
        """
        Recursively performs the "occurs check" during unification.
        If self represents the same variable as other, returns True.
        Else if other is an instance of Expression,
        and self occurs in one or more of other's arguments, returns True.
        Otherwise, returns False.
        """
        if isinstance(other, Variable) and self.__eq__(other):
            return True
        if isinstance(other, Expression) and self.__str__() in other.__str__():
            return True
        return False


class Constant:
    def __init__(self, constant_name):
        """
        Initialize a constant with the given name.
        The name should start with a lower-case letter.
        """
        if constant_name[0].isupper(): raise (Exception("Constant name starting with upper-case!"))
        self.constant_name = constant_name

    def __eq__(self, other):
        """
        Checks whether self and other represent the same constant.
        Two constants are the same if they have the same name.
        """
        return isinstance(other, Constant) and self.constant_name == other.constant_name

    def __ne__(self, other):
        """
        Checks whether self and other represent different constants.
        """
        return not self.__eq__(other)

    def __str__(self):
        """
        Returns a string representation of the constant.
        """
        return self.constant_name

    def __repr__(self):
        """
        Returns a string representation of the constant.
        """
        return str(self)

    def __hash__(self):
        """
        Produces a hash of the constant so it can be stored in a dictionary.
        """
        return str(self).__hash__()


class Expression:
    """
    Represents a compound FOL expression with a top-level function or predicate symbol.
    The "operator" field is the top-level function or predicate symbol.
    The "arguments" field is a list of Constants, Variables, and/or Expression objects.
    """

    def __init__(self, operator, arguments):
        """
        operator: a function or predicate symbol (a string)
        arguments: a list of Constants, Variables, and/or Expression objects.
        """
        self.operator = operator
        self.arguments = arguments

    def __str__(self):
        """
        Returns a string representation of the expression.
        """
        return "%s(%s)" % (
            self.operator,
            ", ".join(map(str, self.arguments)))

    def __repr__(self):
        """
        Returns a string representation of the expression.
        """
        return str(self)

    def __hash__(self):
        """
        Produces a hash of the expression so it can be stored in a dictionary.
        """
        return str(self).__hash__()

    def __eq__(self, other):
        """
        Checks whether self and other represent the same expression.
        Two expressions are the same if they have the same operator
        and each of their respective arguments are the same.
        """
        if not isinstance(other, Expression): return False
        if self.operator != other.operator: return False
        if len(self.arguments) != len(other.arguments): return False
        return all([a1 == a2 for a1, a2 in zip(self.arguments, other.arguments)])

    def __ne__(self, other):
        """
        Checks whether self and other represent different expressions.
        """
        return not self.__eq__(other)


def parse_expression(s):
    """
    Parses a string s and returns a corresponding expression object.
    Assumes arguments are separated by commas and zero or more whitespace.
    """
    l, d, i = [], 0, 0
    op, args = None, []
    for j, c in enumerate(s):
        if c == "(":
            if op is None:
                op = s[:j]
                i = j + 1
            d += 1
        if c == ")":
            if d == 1:
                if j > i: args.append(s[i:j])
                i = j + 1
            d -= 1
        if c == "," and d == 1:
            args.append(s[i:j])
            i = j + 1
        if c == " " and i == j: i += 1

    if op is None:
        if s[0].isupper():
            return Variable(s)
        else:
            return Constant(s)

    return Expression(op, list(map(parse_expression, args)))




In [0]:
# unify_with_occurrence_check(formular1, formular2, mgu = [], trace = False):
# This method takes in four arguments; the two formulas we want to unify,
# a list named mgu(most general unifier) to keep track of the already unified inputs
# and a trace argument that runs the pp() method; this allows us to see/trace the logs for 
# the unification process - meant for testing purposes
#
# unify_variable: This method is executed when one of the formulas entered in 
# (unify_with_occurrence_check) is a variable
#
# substitute(sub, expr): This method applies substituition to sub and expr; which replace on with the other.
#
# occurs_in(var, expr): Checks to see if variable var occurs anywhere in expr
#
# pp(trace, *args): Traces the execution of the unifier, generally used for testing


def unify_with_occurrence_check(formular1, formular2, mgu = [], trace = False):
	#pp(trace, "Unifying expression:", formular1, "with expression:", formular2)
	if mgu is None:
		return None
	elif formular1 == formular2:
		return mgu
	elif isinstance(formular1, Variable):
		return unify_variable(formular1, formular2, mgu, trace)
	elif isinstance(formular2, Variable):
		return unify_variable(formular2, formular1, mgu, trace)
	elif isinstance(formular1, Expression) and isinstance(formular2, Expression):
		if type(formular1) != type(formular2) or formular1.operator != formular2.operator or len(formular1.arguments) != len(formular2.arguments):
			return None
		else:
			for a,b in zip(formular1.arguments, formular2.arguments):
				mgu = unify_with_occurrence_check(a, b, mgu, trace)
			return mgu
	else:
		return None
		
		
		
def substitute(sub, expr):
	"""
    This method applies substituition to sub and expr; which replaces
	on with the other.
    """	
	for s in (x for x in sub if occurs_in(x.variable, expr)):
		if isinstance(expr, Variable):
			expr = s.replacement
		else:
			expr.arguments = [substitute(sub, e) for e in expr.arguments]
	return expr
	
  

def occurs_in(var, expr):
	"""
	Return true if variable var occurs anywhere in expr
	"""
	if var == expr:
		return True
	if not isinstance(expr, Expression):
		return False
	return any([occurs_in(var, e) for e in expr.arguments])
  
  
  
def unify_variable(var, exp, mgu, trace):
	#pp(trace, "Unifying variable:", var, "with expression:", exp)
	for s in (x for x in mgu if x.variable == var):
		return unify_with_occurrence_check(s.replacement, exp, mgu, trace)
	t = substitute(mgu, exp)
	if occurs_in(var, t) and isinstance(t, Expression):
		print("\nCannot unify - infinte loop exception!!!")
		return None
	else:
		s = Substitution(var, t)
		mgu = mgu+[s]
		for q in (x for x in mgu if x.replacement == s.variable):
			mgu.remove(q)
			new  = Substitution(q.variable, s.replacement)
			mgu = mgu+[new]
		for r in (x for x in mgu if isinstance(x.replacement, Expression)):
			mgu.remove(r)
			a = substitute(mgu, r.replacement)
			b = Substitution(r.variable, a)
			mgu = mgu+[b]
		for s in (x for x in mgu if (x.variable == x.replacement)):
			#print("Variable already unified, duplicate deleted!!!")
			mgu.remove(s)
		#pp(trace, "MGU now is: ", ", ".join(map(str, mgu)))
		return mgu




In [21]:
# This script is a utility for running all implemented unification algorithms.
# After running the script in the command prompt, the user can input the two formulas with which they wish to unifier!

def main():

	keep_running = True

	while keep_running:
  
		print("\nPlease enter the first term:")
		t1 = input("-->")
		print("\nPlease enter the second term:")
		t2 = input("-->")
		
	
		#mgu: Most General Unifier
		mgu = unify_with_occurrence_check(parse_expression(t1), parse_expression(t2), trace = False)	
		if mgu is None:
			print("\nno")
		else:
			print("\n")
			print("\n".join(map(str, mgu)))
			print("\nyes")
  
  
		print("\n\n>>> Do you want to run unifier again? (Y/N)")
		re_run = input("--> ")

		if re_run != "y" and re_run != "Y":
			keep_running = False
		
		
# Run main
if __name__ == "__main__":
	main()


Please enter the first term:
-->p(B,A)

Please enter the second term:
-->p(A,B)


B = A

yes


>>> Do you want to run unifier again? (Y/N)
--> y

Please enter the first term:
-->f(X,g(X))

Please enter the second term:
-->f(g(Y),Y)

Cannot unify - infinte loop exception!!!

no


>>> Do you want to run unifier again? (Y/N)
--> n
